In [2]:
# import all necessary libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

In [3]:
# load data
SEs = pd.read_csv('stock_ex.csv', index_col=0, names=['SE', 'city','iso3'])
adr = pd.read_csv('Compustat_Addresses.csv')[['datadate','isin','conm','fic','city']]
adr_US = pd.read_csv('Compustat_Addresses_US.csv')[['datadate','cusip','conm','fic','city']]
smi_adr = pd.read_csv('smi_adr.csv')
sec = pd.read_stata('Securities.dta')
cities = pd.read_csv('worldcities.csv')
country_codes = pd.read_csv('cc.csv', sep='\t',header=None)[2].values

/home/svkohler/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# flags
pre_calc = True

In [5]:
def firstx(x,length):
        return x[:length]
# clean up adress data table such that there is one line per ISIN/CUSIP
pre_calc = True
if not pre_calc:
    # drop all rows with no isin
    adr =adr.drop(adr[adr['isin'].isnull()].index)
    idx = []
    for isin in tqdm(adr['isin'].unique()):
        idx.append(adr[adr['isin']==isin].index.values[-1])
    adr = adr.loc[idx]
    adr.to_csv('adr_c.csv', index=False)

if not pre_calc:
    # drop all rows with no cusip
    adr_US = adr_US.drop(adr_US[adr_US['cusip'].isnull()].index)
    idx = []
    for cusip in tqdm(adr_US['cusip'].unique()):
        idx.append(adr_US[adr_US['cusip']==cusip].index.values[-1])
    adr_US = adr_US.loc[idx]
    cusip6 = [firstx(x,6) for x in adr_US['cusip']]
    cusip8 = [firstx(x,8) for x in adr_US['cusip']]
    adr_US['cusip6'] = cusip6
    adr_US['cusip8'] = cusip8
    adr_US[['cusip6','cusip8','conm','fic','city']].to_csv('adr_US_c.csv', index=False)

# load clean adresses
adr_c = pd.read_csv('adr_c.csv')
adr_US_c = pd.read_csv('adr_US_c.csv')

# add smi adresses
adr_c = adr_c.append(smi_adr)

In [6]:
# merge stock exchanges with cities to get coordinates. (problem multiple Washington, USA)
SEs_geo = SEs.merge(cities, how='left', left_on=['city','iso3'], right_on=['city_ascii','iso3']).iloc[:,0:8]
SEs_geo.head()

,SE,city_x,iso3,city_y,city_ascii,lat,lng,country
0,New York Stock Exchange (NYSE),New York,USA,New York,New York,40.6943,-73.9249,United States
1,Germany,Frankfurt,DEU,Frankfurt,Frankfurt,50.1136,8.6797,Germany
2,London SETS,London,GBR,London,London,51.5072,-0.1275,United Kingdom
3,Eurex,Frankfurt,DEU,Frankfurt,Frankfurt,50.1136,8.6797,Germany
4,SIX Swiss Exchange,Zurich,CHE,Zürich,Zurich,47.3786,8.5400,Switzerland


In [7]:
# get country codes
cc = [firstx(x,2) for x in sec['isin']]
sec['country_code'] = cc
sec['country_code'][-sec['country_code'].isin(country_codes)] = 'invalid'

/tmp/ipykernel_3141/4130974182.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sec['country_code'][-sec['country_code'].isin(country_codes)] = 'invalid'


In [8]:
pre_calc = True
# isolate Options
opt = sec[sec['gr_details']=='Options']
# define a function to separate the first term of the symbol
def cut(x):
    return x.split(' ', 1)[0]
# get a list with all symbols (first part)
symbols_opt = [cut(x) for x in opt['name'].values]
cc_opt = opt['country_code'].values
tup_opt = list(zip(symbols_opt,cc_opt))
if not pre_calc:
    # collect a dictionary with: key=symbol, value=isin. Map symbol to isin if unambiguous. Take advantage of redundant symbols.
    coll = dict()
    for i in tqdm(set(tup_opt)):
        if i[0] in coll:
            if coll[i[0]] == 'ambiguous':
                # select subpart of securities with given symbol
                sub = sec[sec['symbol']==i[0]]
                sub_cc = sub[sub['country_code']==i[1]]
                # check if only one ISIN is present
                if len(sub['isin'].unique())==1:
                    coll[i[0]] = sub['isin'].unique()[0]
                elif i[1] != 'invalid' and len(sub['isin'].unique())>1 and len(sub_cc['isin'].unique())==1:
                    coll[i[0]] = sub_cc['isin'].unique()[0]
                else:
                    coll[i[0]] = 'ambiguous'
        else:
            # select subpart of securities with given symbol
                sub = sec[sec['symbol']==i[0]]
                sub_cc = sub[sub['country_code']==i[1]]
                # check if only one ISIN is present
                if len(sub['isin'].unique())==1:
                    coll[i[0]] = sub['isin'].unique()[0]
                elif i[1] != 'invalid' and len(sub['isin'].unique())>1 and len(sub_cc['isin'].unique())==1:
                    coll[i[0]] = sub_cc['isin'].unique()[0]
                else:
                    coll[i[0]] = 'ambiguous'
    pickle.dump(coll, open('isin_ambig_dict.p', 'wb'))

In [9]:
pre_calc = True
# isolate Derivatives
der = sec[sec['gr_details']=='Derivatives']
# get a list with all symbols (first part)
symbols_der = [cut(x) for x in der['name'].values]
cc_der = der['country_code'].values
tup_der = list(zip(symbols_der,cc_der))
if not pre_calc:
    # collect a dictionary with: key=symbol, value=isin. Map symbol to isin if unambiguous. Take advantage of redundant symbols.
    coll = dict()
    for i in tqdm(set(tup_der)):
        if len(i[0])<5:
            if i[0] in coll:
                if coll[i[0]] == 'ambiguous':
                    # select subpart of securities with given symbol
                    sub = sec[sec['symbol']==i[0]]
                    sub_cc = sub[sub['country_code']==i[1]]
                    # check if only one ISIN is present
                    if len(sub['isin'].unique())==1:
                        coll[i[0]] = sub['isin'].unique()[0]
                    elif i[1] != 'invalid' and len(sub['isin'].unique())>1 and len(sub_cc['isin'].unique())==1:
                        coll[i[0]] = sub_cc['isin'].unique()[0]
                    else:
                        coll[i[0]] = 'ambiguous'
            else:
                # select subpart of securities with given symbol
                    sub = sec[sec['symbol']==i]
                    sub_cc = sub[sub['country_code']==i[1]]
                    # check if only one ISIN is present
                    if len(sub['isin'].unique())==1:
                        coll[i[0]] = sub['isin'].unique()[0]
                    elif i[1] != 'invalid' and len(sub['isin'].unique())>1 and len(sub_cc['isin'].unique())==1:
                        coll[i[0]] = sub_cc['isin'].unique()[0]
                    else:
                        coll[i[0]] = 'ambiguous'
        else:
            coll[i[0]] = 'unknown'
    pickle.dump(coll, open('isin_ambig_dict_der.p', 'wb'))

In [10]:
# load the precalculated object
coll_opt = pickle.load(open('isin_ambig_dict.p', 'rb'))
coll_der = pickle.load(open('isin_ambig_dict_der.p', 'rb'))

In [11]:
import warnings
warnings.filterwarnings("ignore")
# map new ISINs to Option securities. Do the same for CUSIP. When ISIN is ambiguous then so is the CUSIP.
isins = []
for s in tqdm(range(len(symbols_opt))):
    symbol = symbols_opt[s]
    isins.append(coll_opt[symbol])
opt['isin'] = isins
#opt['cusip'] = [extract_cusip6(x) for x in isins]
# simplify the name by replacing it with the symbol
opt['name'] = symbols_opt
# replace in securities table
sec[sec['gr_details']=='Options'] = opt

isins = []
for s in tqdm(range(len(symbols_der))):
    symbol = symbols_der[s]
    isins.append(coll_der[symbol])
der['isin'] = isins
#der['cusip'] = [extract_cusip6(x) for x in isins]
# simplify the name by replacing it with the symbol
der['name'] = symbols_der
# replace in securities table
sec[sec['gr_details']=='Derivatives'] = der
sec.head(20)

100%|██████████| 2070829/2070829 [00:00<00:00, 2179199.15it/s]


,security_id,isin,currency,stock,symbol,name,cfi,gr,gr_details,country_code
0,-1,,,,MISSING,SECURITY MISSING,,,,invalid
1,0,,,,,DELETED SECURITIES,,,,invalid
2,1,US86800C1045,United States Dollar,New York Stock Exchange (NYSE),STP,SUNTECH POWER HOLDING ADR REP,ESXXXX,Shares,Foreign Shares,US
3,10,unknown,Euro,Germany,DX22SW,DX22SW,RWXXXX,Structured products,Derivatives,DE
4,100,GB0004161021,British Pound,London SETS,HAS,HAYS ORD,ESVUFR,Shares,Foreign Shares,GB
5,1000,ambiguous,Euro,Eurex,ODAX JAN3 7300P,ODAX,OPEICS,Structured products,Options,invalid
6,10000,unknown,Euro,Germany,CK06AN,CK06AN,RWXXXX,Structured products,Derivatives,DE
7,100000,unknown,Euro,Germany,CK6HRR,CK6HRR,RWXXXX,Structured products,Derivatives,DE
8,1000000,ambiguous,Swiss Franc,Eurex,FIN MAR7 800P,FIN,OPASPS,Structured products,Options,invalid
9,1000001,ambiguous,Euro,Eurex,RWE DEC6 11.5P,RWE,OPASPS,Structured products,Options,invalid


In [12]:
# extract the country code and the 6-digit-CUSIP from the ISIN of the securities
def extract_cusip6(x):
    if firstx(x,2) in ['US', 'CA']:
        return x[2:8]
    else:
        return 'NoCUSIP'
def extract_cusip8(x):
    if firstx(x,2) in ['US', 'CA']:
        return x[2:10]
    else:
        return 'NoCUSIP'
cusip6 = [extract_cusip6(x) for x in sec['isin']]
cusip8 = [extract_cusip8(x) for x in sec['isin']]
sec['cusip6'] = cusip6
sec['cusip8'] = cusip8
sec.head(20)

,security_id,isin,currency,stock,symbol,name,cfi,gr,gr_details,country_code,cusip6,cusip8
0,-1,,,,MISSING,SECURITY MISSING,,,,invalid,NoCUSIP,NoCUSIP
1,0,,,,,DELETED SECURITIES,,,,invalid,NoCUSIP,NoCUSIP
2,1,US86800C1045,United States Dollar,New York Stock Exchange (NYSE),STP,SUNTECH POWER HOLDING ADR REP,ESXXXX,Shares,Foreign Shares,US,86800C,86800C10
3,10,unknown,Euro,Germany,DX22SW,DX22SW,RWXXXX,Structured products,Derivatives,DE,NoCUSIP,NoCUSIP
4,100,GB0004161021,British Pound,London SETS,HAS,HAYS ORD,ESVUFR,Shares,Foreign Shares,GB,NoCUSIP,NoCUSIP
5,1000,ambiguous,Euro,Eurex,ODAX JAN3 7300P,ODAX,OPEICS,Structured products,Options,invalid,NoCUSIP,NoCUSIP
6,10000,unknown,Euro,Germany,CK06AN,CK06AN,RWXXXX,Structured products,Derivatives,DE,NoCUSIP,NoCUSIP
7,100000,unknown,Euro,Germany,CK6HRR,CK6HRR,RWXXXX,Structured products,Derivatives,DE,NoCUSIP,NoCUSIP
8,1000000,ambiguous,Swiss Franc,Eurex,FIN MAR7 800P,FIN,OPASPS,Structured products,Options,invalid,NoCUSIP,NoCUSIP
9,1000001,ambiguous,Euro,Eurex,RWE DEC6 11.5P,RWE,OPASPS,Structured products,Options,invalid,NoCUSIP,NoCUSIP


In [13]:
# merge the adress dataframe with the cities dataframe to combine ISIN/CUSIP with longitude and latitude
# Results in two merged tables once for CUSIP and once for ISIN.
m=adr_c.merge(cities, left_on=['city', 'fic'], right_on=['city_ascii', 'iso3'], how='inner')[['conm','isin', 'iso3', 'lat','lng', 'city_x']]
m_US=adr_US_c.merge(cities, left_on=['city', 'fic'], right_on=['city_ascii', 'iso3'], how='inner')[['conm','cusip6','cusip8', 'iso3', 'lat','lng', 'city_x']]

In [14]:
# take the mean of lat, lng for ISINs/CUSIPs with multiple coordinate information (possible data contamination)
counts = m['isin'].value_counts()
for i in tqdm(range(len(counts))):
    if counts.values[i]>1:
        sub = m[m['isin']==counts.index[i]]
        mean_lat = sub['lat'].mean()
        mean_lng = sub['lng'].mean()
        sub.iloc[0,4] = mean_lat
        sub.iloc[0,5] = mean_lng
        # drop multiple
        m.drop(m[m['isin']==counts.index[i]].index, inplace=True)
        # replace by mean
        m=m.append(sub.iloc[0,:])

m_US_cusip8 = m_US.copy()
counts = m_US_cusip8['cusip8'].value_counts()
for i in tqdm(range(len(counts))):
    if counts.values[i]>1:
        sub = m_US_cusip8[m_US_cusip8['cusip8']==counts.index[i]]
        mean_lat = sub['lat'].mean()
        mean_lng = sub['lng'].mean()
        sub.iloc[0,4] = mean_lat
        sub.iloc[0,5] = mean_lng
        # drop multiple
        m_US_cusip8.drop(m_US_cusip8[m_US_cusip8['cusip8']==counts.index[i]].index, inplace=True)
        # replace by mean
        m_US_cusip8=m_US_cusip8.append(sub.iloc[0,:])

m_US_cusip6 = m_US.copy()
counts = m_US_cusip6['cusip6'].value_counts()
for i in tqdm(range(len(counts))):
    if counts.values[i]>1:
        sub = m_US_cusip6[m_US_cusip6['cusip6']==counts.index[i]]
        mean_lat = sub['lat'].mean()
        mean_lng = sub['lng'].mean()
        sub.iloc[0,4] = mean_lat
        sub.iloc[0,5] = mean_lng
        # drop multiple
        m_US_cusip6.drop(m_US_cusip6[m_US_cusip6['cusip8']==counts.index[i]].index, inplace=True)
        # replace by mean
        m_US_cusip6=m_US_cusip6.append(sub.iloc[0,:])

100%|██████████| 9126/9126 [00:21<00:00, 427.58it/s]


In [15]:
# merge the securities with the geographic information. Here again once for ISIN/CUSIP
sec_geo = sec.merge(m, how='left', on=['isin'])
sec_geo_cusip8 = sec.merge(m_US_cusip8, how='left', on=['cusip8'])#.iloc[:,np.r_[0:12,14:18]].rename(columns={'cusip6_x':'cusip6', 'city_x':'city'})

In [19]:
# replace those rows where the city value for the CUSIP-merge is NOT null with the values from the CUSIP merge
sec_geo.loc[-sec_geo_cusip8['city_x'].isnull()] = sec_geo_cusip8.loc[-sec_geo_cusip8['city_x'].isnull()]

In [20]:
# Due to several problems with the data some symbols will be filled in manually

# NY
NY = sec_geo[sec_geo['name'].isin(('EMINI','Emini','SPY','QQQ','MNQ','DJIA','.DJIA','NDX','IWM','DIA','SPX','TQQQ','C'))]
NY['iso3'] = 'USA'
NY['lat'] = cities[cities['city_ascii']=='New York']['lat'].values[0]
NY['lng'] = cities[cities['city_ascii']=='New York']['lng'].values[0]
NY['city_x'] = cities[cities['city_ascii']=='New York']['city'].values[0]
sec_geo[sec_geo['name'].isin(('EMINI','Emini','SPY','QQQ','MNQ','DJIA','.DJIA','NDX','IWM','DIA','SPX','TQQQ','C'))] = NY

# MVIEW
MVIEW = sec_geo[sec_geo['name'].isin(('GOOG','GOOGL'))]
MVIEW['iso3'] = 'USA'
MVIEW['lat'] = cities[cities['city_ascii']=='Mountain View']['lat'].values[0]
MVIEW['lng'] = cities[cities['city_ascii']=='Mountain View']['lng'].values[0]
MVIEW['city_x'] = cities[cities['city_ascii']=='Mountain View']['city'].values[0]
sec_geo[sec_geo['name'].isin(('GOOG','GOOGL'))] = MVIEW

# FRANKFURT
FRANKFURT = sec_geo[sec_geo['name'].isin(('DAX','ODAX','SX5E'))]
FRANKFURT['iso3'] = 'DEU'
FRANKFURT['lat'] = cities[cities['city_ascii']=='Frankfurt']['lat'].values[0]
FRANKFURT['lng'] = cities[cities['city_ascii']=='Frankfurt']['lng'].values[0]
FRANKFURT['city_x'] = cities[cities['city_ascii']=='Frankfurt']['city'].values[0]
sec_geo[sec_geo['name'].isin(('DAX','ODAX','SX5E'))] = FRANKFURT

# ZURICH
ZURICH = sec_geo[sec_geo['name'].isin(('SMI','OSMI','CSGN','BAEN'))]
ZURICH['iso3'] = 'CHE'
ZURICH['lat'] = cities[cities['city_ascii']=='Zurich']['lat'].values[0]
ZURICH['lng'] = cities[cities['city_ascii']=='Zurich']['lng'].values[0]
ZURICH['city_x'] = cities[cities['city_ascii']=='Zurich']['city'].values[0]
sec_geo[sec_geo['name'].isin(('SMI','OSMI','CSGN','BAEN'))] = ZURICH

# VEVEY
VEVEY = sec_geo[sec_geo['name'].isin(('NESN','NESTLE'))]
VEVEY['iso3'] = 'CHE'
VEVEY['lat'] = cities[cities['city_ascii']=='Vevey']['lat'].values[0]
VEVEY['lng'] = cities[cities['city_ascii']=='Vevey']['lng'].values[0]
VEVEY['city_x'] = cities[cities['city_ascii']=='Vevey']['city'].values[0]
sec_geo[sec_geo['name'].isin(('NESN','NESTLE'))] = VEVEY

# GENEVA
GENEVA = sec_geo[sec_geo['name'].isin(('GIVN','GIVAUDAN'))]
GENEVA['iso3'] = 'CHE'
GENEVA['lat'] = cities[cities['city_ascii']=='Geneva']['lat'].values[0]
GENEVA['lng'] = cities[cities['city_ascii']=='Geneva']['lng'].values[0]
GENEVA['city_x'] = cities[cities['city_ascii']=='Geneva']['city'].values[0]
sec_geo[sec_geo['name'].isin(('GIVN','GIVAUDAN'))] = GENEVA

# LONDON
LONDON = sec_geo[sec_geo['name'].isin(('VXX','UVXY','XAG-USD'))]
LONDON['iso3'] = 'GBR'
LONDON['lat'] = cities[cities['city_ascii']=='London']['lat'].values[0]
LONDON['lng'] = cities[cities['city_ascii']=='London']['lng'].values[0]
LONDON['city_x'] = cities[cities['city_ascii']=='London']['city'].values[0]
sec_geo[sec_geo['name'].isin(('VXX','UVXY','XAG-USD'))] = LONDON

In [21]:
# parallel to fill all the remaining NaN-values a separate table is produced where the securities are merged based on stock exchange location.
sec_SE = sec.merge(SEs_geo, how='left', left_on='stock', right_on='SE')
sec_SE.head()

,security_id,isin,currency,stock,symbol,name,cfi,gr,gr_details,country_code,cusip6,cusip8,SE,city_x,iso3,city_y,city_ascii,lat,lng,country
0,-1,,,,MISSING,SECURITY MISSING,,,,invalid,NoCUSIP,NoCUSIP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,,,,,DELETED SECURITIES,,,,invalid,NoCUSIP,NoCUSIP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,US86800C1045,United States Dollar,New York Stock Exchange (NYSE),STP,SUNTECH POWER HOLDING ADR REP,ESXXXX,Shares,Foreign Shares,US,86800C,86800C10,New York Stock Exchange (NYSE),New York,USA,New York,New York,40.6943,-73.9249,United States
3,10,unknown,Euro,Germany,DX22SW,DX22SW,RWXXXX,Structured products,Derivatives,DE,NoCUSIP,NoCUSIP,Germany,Frankfurt,DEU,Frankfurt,Frankfurt,50.1136,8.6797,Germany
4,100,GB0004161021,British Pound,London SETS,HAS,HAYS ORD,ESVUFR,Shares,Foreign Shares,GB,NoCUSIP,NoCUSIP,London SETS,London,GBR,London,London,51.5072,-0.1275,United Kingdom


In [22]:
# as a last step fill in the coordinates of the missing securites with coordinates of their stock exchange
# copy previous dataframe
sec_geo_se = sec_geo
# mask to fill in remaining values
mask = sec_geo['lat'].isnull()
# columnwise fill lat/lng with values from stock exchange locations
sec_geo_se['lat'].loc[mask] = sec_SE['lat'].loc[mask]
sec_geo_se['lng'].loc[mask] = sec_SE['lng'].loc[mask]
sec_geo_se['city_x'].loc[mask] = sec_SE['city_x'].loc[mask]
sec_geo_se[['security_id','isin','stock','name','gr','gr_details','cusip6','cusip8','city_x','iso3','lat','lng']].to_csv('distance.csv')